<a href="https://colab.research.google.com/github/nirkoren1/Chord_Transformer/blob/master/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 17.4 MB/s 


In [26]:
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib import style

In [ ]:
path_to_dictionary = r"/content/tokens"
with open(path_to_dictionary, "rb") as f:
  dictionary = pickle.load(f)
dictionary

In [31]:
path_to_data = r"/content/data.pickle"
with open(path_to_data, "rb") as f:
  data = pickle5.load(f)
data

,genres,popularity,chords,training_data
0,"[canadian pop, pop, post-teen pop]",100,"[G, G/B, B, C, G, G, G/B, B, C, G, G, Em, C, G...","[<start-genres>, canadian pop, pop, post-teen ..."
1,"[canadian pop, pop, post-teen pop]",100,"[F#m7, D2, F#m7, D2, F#m7, D2, E, F#m7, A/C#, ...","[<start-genres>, canadian pop, pop, post-teen ..."
2,"[canadian pop, pop, post-teen pop]",100,"[Em, D, C, C, D, Em, Em, D, C, C, D, Em, Em, D...","[<start-genres>, canadian pop, pop, post-teen ..."
3,"[canadian pop, pop, post-teen pop]",100,"[Em, Bm, Am, C, Em, Bm, Am, C, Em, Bm, Am, C, ...","[<start-genres>, canadian pop, pop, post-teen ..."
4,"[canadian pop, pop, post-teen pop]",100,"[Gm, Dm, C, C, Gm, Dm, C, C, Gm, Dm, C, C, Gm,...","[<start-genres>, canadian pop, pop, post-teen ..."
...,...,...,...,...
135777,"[classic rock, folk, folk rock, melancholia, m...",76,"[C, Am, F, G, C, Am, F, G, C, Am, F, G, C, Am,...","[<start-genres>, classic rock, folk, folk rock..."
135778,"[classic rock, folk, folk rock, melancholia, m...",76,"[B, E, B, E, B, E, F#, E, B, F#, E, B, E, B, E...","[<start-genres>, classic rock, folk, folk rock..."
135780,"[classic rock, folk, folk rock, melancholia, m...",76,"[Dm, C, Dm, F, Bb, F, Bb, F, Bb, F, Dm, F, C, ...","[<start-genres>, classic rock, folk, folk rock..."
135781,"[classic rock, folk, folk rock, melancholia, m...",76,"[G, Em, C, G, G, Am, C, D, G, Em, C, G, G, Am,...","[<start-genres>, classic rock, folk, folk rock..."


In [34]:
def tokenize(batch):
  out = []
  for sample in batch:
    ids = []
    for token in sample:
      ids.append(dictionary[token])
    out.append(sample)
  return out

example = data["training_data"][0]
print(tokenize([example]))

[['<start-genres>', 'canadian pop', 'pop', 'post-teen pop', '<start>', 'G', 'G/B', 'B', 'C', 'G', 'G', 'G/B', 'B', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'G/B', 'B', 'C', 'G', 'G', 'G/B', 'B', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'B', 'Bm', 'C', 'Bm', 'C', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'G', 'G', 'Em', 'C', 'C', 'G', 'G', 'G', 'Em', 'C', 'G', '<end>']]
